# Intrusion Detection Computer Vision System

## 1. Introduction

The given tasks are the following:
- For each frame of an input video, the system needs to display the found blobs (either by coloring them on a black background or by showing the countours on top of the original video) and produce a textual output listing the found blobs and their meaningful features.
- The system is required to discriminate between a present blob and a false one originated by the removal of an object in the background.

The proposed solution uses the developed library `intrusiondetection` and this notebook shows the step-by-step operations computed to achieve the final outputs alongside considerations over the made choices. 

In [ ]:
import numpy as np
import cv2
from matplotlib import pyplot as plt

from intrusiondetection.parameters import ParameterList
from intrusiondetection.utility import distance_euclidean
from intrusiondetection.model import MorphOp, MorphOpsSet, Video, Background

# Only for jupyter notebook visualization
%matplotlib inline 

video = Video("rilevamento-intrusioni-video.avi")

### 1.1. Considerations

The given video presents the following characteristics:
- 12 fps
- ~41s
- 320x240 pixels
- 8 bit/pixel (256 gray levels)

Osservando il contenuto si possono notare i seguenti dettagli:
- Il video presenta del rumore, perciò sarà necessario utilizzare degli operatori di bin morph per migliorare il risultato ottenuto
- A primo impatto si osserva che ci sono dei cambi di luce consistenti nel corso del video: per questo motivo si intuisce che adoperare un background dinamico possa migliorare il risultato, la tecnica è utilizzabile grazie anche al numero di FPS non estremamente ridotto.
- La persona nel corso del video cambia la propria velocità, per questo motivo sarà necessario ottimizzare l'adaptation rate del background

## 2. Background Estimation

### 2.1. Initial Background

To estimate a good background for the scene, it's necessary to perform an interpolation between some chosen frames of the video.

- The different interpolation functions considered are `np.mean` and `np.median`.
- The amount of initial frames is tuned based on a tradeoff between the smallest and most stable number, therefore the considered values are 60, 80, 100

In [ ]:
video_input_path = "rilevamento-intrusioni-video.avi"

bg1 = Background(video_input_path, np.median, 60)
bg2 = Background(video_input_path, np.median, 80)
background = Background(video_input_path, np.median, 100)
bg4 = Background(video_input_path, np.mean, 60)
bg5 = Background(video_input_path, np.mean, 80)
bg6 = Background(video_input_path, np.mean, 100)


plt.figure(figsize=(20, 10))
plt.subplot(1, 3, 1)
bg1.show('image', show=False)
plt.subplot(1, 3, 2)
bg2.show('image', show=False)
plt.subplot(1, 3, 3)
background.show('image', show=False)
plt.show()

plt.figure(figsize=(20, 10))
plt.subplot(1, 3, 1)
bg4.show('image', show=False)
plt.subplot(1, 3, 2)
bg5.show('image', show=False)
plt.subplot(1, 3, 3)
bg6.show('image', show=False)
plt.show()

TODO Considerations

### 2.2. Background update
The two main approaches to obtain a dynamic background are `blind` and `selective`.

The following section shows an example of a blind background computed using an adaption rate of `0.1`

In [ ]:
blind_backgrounds = video.process_backgrounds('blind', background, 0.1)
curr_background = blind_backgrounds[300]
curr_background.show('image')

For the selective approach, there are also threshold and distance function parameters to compute the subtraction between the frame and the background, furthermore, some binary morphology operators are applied to get rid of most of the noise.

In [ ]:
selective_backgrounds = video.process_backgrounds('selective', background, 0.1, 37, distance_euclidean, MorphOpsSet(
    MorphOp(cv2.MORPH_CLOSE, (3,3), iterations=1),
    MorphOp(cv2.MORPH_OPEN, (3,3), iterations=2), 
    MorphOp(cv2.MORPH_DILATE, (25,10)), 
    MorphOp(cv2.MORPH_ERODE, (15,5))
))
curr_background = selective_backgrounds[400]
curr_background.show(['subtraction', 'mask_raw', 'mask_refined', 'image'])

A selective background clearly obtains better results, it is therefore the preferred method.

## 3. Change Detection

### 3.1. Background Subtraction

To perform the change detection the first step is to subtract the current frame with respect to the corresponding background:

In [ ]:
curr_frame = video.frames[400]
curr_frame.apply_change_detection(curr_background, 37, distance_euclidean)
curr_frame.show(['subtraction', 'mask_raw'])

### 3.2. Binary Morphology

We then use binary morphology operators:
- 3x3 Opening to remove the noise
- 5x5 Closing 
- 40x3 Opening 

In [ ]:
curr_frame.apply_morphology_operators(MorphOpsSet(
    MorphOp(cv2.MORPH_OPEN, (3,3))
))
plt.subplot(1, 3, 1)
curr_frame.show('mask_refined', show=False)

curr_frame.apply_morphology_operators(MorphOpsSet(
    MorphOp(cv2.MORPH_OPEN, (3,3)),
    MorphOp(cv2.MORPH_CLOSE, (5,5), iterations=3)
))
plt.subplot(1, 3, 2)
curr_frame.show('mask_refined', show=False)

curr_frame.apply_morphology_operators(MorphOpsSet(
    MorphOp(cv2.MORPH_OPEN, (3,3)),
    MorphOp(cv2.MORPH_CLOSE, (5,5), iterations=3),
    MorphOp(cv2.MORPH_CLOSE, (40,3))
))
plt.subplot(1, 3, 3)
curr_frame.show('mask_refined', show=False)

plt.show()

## 4. Blob Analysis

### 4.1. Blob Labeling

The labeling of the obtained image is performed using the `cv2.connectedComponents` function.

In [ ]:
curr_frame.apply_blob_analysis([])
curr_frame.show('blobs_filled')

### 4.2. Selected Features

The meaningful features considered are:
- Area
- Perimeter
- Barycentre

### 4.3. Blob Similarity
It is necessary to keep continuity between the blob labeling in subsequential frames, therefore, a dissimilarity function is computed to seek for correspondances between the current and previous blobs.
The blobs are displayed with the label printed on their barycentre.

The function is defined as: TODO

The dissimilarity function holds a threshold parameter to define an upper bound for a maximum dissimilarity above which two blobs are always considered different.

TODO: Mostrare dissimilarity tra tutti gli oggetti per dare un'idea della media?

In [ ]:
param_bag = ParameterList({
    "input_video": "rilevamento-intrusioni-video.avi",
    "output_directory": "output",
    "threshold": [37],
    "distance": [distance_euclidean],
    "morph_ops": [
        MorphOpsSet(MorphOp(cv2.MORPH_OPEN, (3,3)), MorphOp(cv2.MORPH_CLOSE, (5,5), iterations=3), MorphOp(cv2.MORPH_CLOSE, (40,3)))#MorphOp(cv2.MORPH_CLOSE, (10,3), iterations=3))
    ],
    "background_threshold": [37],
    "background_distance": [distance_euclidean],
    "background_morph_ops": [
        MorphOpsSet(MorphOp(cv2.MORPH_CLOSE, (3,3), iterations=1),MorphOp(cv2.MORPH_OPEN, (3,3), iterations=2), MorphOp(cv2.MORPH_DILATE, (25,10)), MorphOp(cv2.MORPH_ERODE, (15,5)))
    ],
    "alpha": [0.1],
    "background": {
        "frames": [100],
        "interpolation": [np.median]
    }
})

for p in param_bag:
    params = p

prev_frame = video.frames[399]
prev_frame.intrusion_detection(selective_backgrounds[399], params, [])

curr_frame.apply_blob_analysis(prev_frame.blobs)
#TODO curr_frame.apply_blob_labeling(dissimilarity_threshold)
#TODO curr_frame.show('blobs_labeled
plt.subplot(1, 2, 1)
prev_frame.show('blobs_filled', show=False)
plt.subplot(1, 2, 2)
curr_frame.show('blobs_filled', show=False)
plt.show()

### 4.4. Classification 
To classify the blobs, a function of their features is computed.
The function is defined as: TODO

In [ ]:
#TODO curr_frame.apply_classification(classification_threshold)
#TODO curr_frame.show('blobs_classified')

## 5. True/False Object Recognition

To apply the object recognition the selected approach is the evaluation of the edges strength, such that, when the contours of a found blob presents sharp edges in the original image, it is considered to be present, otherwise it is labeled as fake.

The computation is performed by using a Sobel operator returning a smoothed edge score not taking into account the large amount of noise present in the original frame. 

In [ ]:
#TODO curr_frame.apply_object_recognition(recognition_threshold)
#TODO curr_frame.show('blobs_refined')

## 6. Output Generation

### 6.1. Text Output

A CSV file is then generated, for each frame the following informations are stored:
- Frame Index
- Number of Detected Objects
- Then a row for each detected object is printed listing this features:
   - Object Label
   - Area
   - Perimeter
   - Classification

### 6.2. Video Output

The graphical output shows the contours of the found objects, the color of the contour depends on the object classification:
- Person: Blue
- True Object: Green
- Fake Object: Red

In [ ]:
curr_frame.show('blobs_contours')